<a href="https://colab.research.google.com/github/bflatts/UCDPA_BrianFlaherty/blob/main/BrianFlaherty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
import pandas as pd


In [ ]:
# Variables
DIR = '/content/drive/MyDrive/Colab Notebooks/'
FILENAME = DIR + 'cleaned_5250.csv'

In [ ]:
# Reading CSV file
data = pd.read_csv(FILENAME, header = 0)


In [ ]:
# Sorting
number_to_print = 5

print(f'The furtheset {number_to_print} exoplanets from the sun are:')          #Prints a note
farthest_from_sun = data.sort_values('distance', ascending=False )              #sorts the dataset by distance
print(farthest_from_sun[['name']].head(number_to_print).to_string(index=False)) #prints out the first number of names



The furtheset 5 exoplanets from the sun are:
                 name
           SWEEPS-4 b
          SWEEPS-11 b
 KMT-2021-BLG-1077L b
 KMT-2021-BLG-1077L c
OGLE-2015-BLG-0051L b


In [ ]:
# indexing
data_ind = data.set_index('name')
print(data_ind[['distance','planet_type']].head())

                      distance planet_type
name                                      
11 Comae Berenices b     304.0   Gas Giant
11 Ursae Minoris b       409.0   Gas Giant
14 Andromedae b          246.0   Gas Giant
14 Herculis b             58.0   Gas Giant
16 Cygni B b              69.0   Gas Giant


In [ ]:
# grouping
print('Table showing smallest, largest, and avergage planet size per planet type')
data.groupby('planet_type')['radius_multiplier'].agg(['min', 'max', 'mean'])

Table showing smallest, largest, and avergage planet size per planet type


,min,max,mean
planet_type,,,
Gas Giant,0.225,6.900,1.160728
Neptune-like,0.200,2.233,0.493737
Super Earth,0.201,2.230,1.487342
Terrestrial,0.259,1.800,0.824062
Unknown,NaN,NaN,NaN


In [ ]:
# Checking for duplication and missing values
missing = data[['name']].isnull().sum().to_string(index=False)                  # counts the cells in name that are null, and converts them to string
duplicate = len(data['name'])-len(data['name'].drop_duplicates())               # counts the duplicates in name column

#print message
print('there are : ' + missing + ' missing names' )
print('there are : ' + str(duplicate) + ' duplicated names' )



there are : 0 missing names
there are : 0 duplicated names


In [ ]:
#split data, all entries before 2000
data_before_2000 = data[data['discovery_year'] < 2000]

#split data, all entries after 1999
data_after_1999 = data[data['discovery_year'] > 1999]

print(str(data_before_2000.shape)+'   <- before 2000')
print(str(data_after_1999.shape)+' <- after 1999')
print(str(data.shape)+ ' <- original')

#merge the split parts
data_merge = data_after_1999.merge(data_before_2000, on='name', how='left', suffixes=('_aft99', '_bfr2000'))

print(str(data_merge.shape)+' <- merged')

#concat the split parts
data_concat = pd.concat([data_before_2000,data_after_1999])
print(str(data_concat.shape)+' <- concatenated')

(30, 13)   <- before 2000
(5220, 13) <- after 1999
(5250, 13) <- original
(5220, 25) <- merged
(5250, 13) <- concatenated
